In [1]:
import numpy as np
from intfeat import StrumLiouvilleTransformer, CurvatureSpec
import optuna
from ucimlrepo import fetch_ucirepo
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import SplineTransformer, KBinsDiscretizer, StandardScaler

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Generate dataset

In [3]:
dataset = fetch_ucirepo(id=332)
float_feat_cols = [
 'data_channel_is_world',
 'weekday_is_sunday',
 'kw_avg_max',
 'is_weekend',
 'kw_max_avg',
 'global_rate_positive_words',
 'abs_title_sentiment_polarity',
 'self_reference_avg_sharess',
 'max_positive_polarity',
 'avg_positive_polarity']
int_feat_cols = ['kw_min_max', 'n_tokens_title']
all_features = float_feat_cols + int_feat_cols

X = dataset.data.features
y = dataset.data.targets

X.columns = X.columns.map(lambda col: col.strip())
X = X[all_features]
X = X.astype({col: np.int32 for col in int_feat_cols})

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [5]:
num_funcs = 5

In [6]:
def make_spline_pipeline(knots='uniform', degree=3, num_funcs=num_funcs):
    n_knots = num_funcs - (degree - 1)
    return make_pipeline(
        SplineTransformer(n_knots=n_knots, knots=knots, degree=degree),
        RidgeCV(alphas=np.geomspace(1e-3, 1e3, 20))
    )

In [7]:
def make_bins_pipeline(strategy='uniform', num_funcs=num_funcs):
    return make_pipeline(
        KBinsDiscretizer(n_bins=num_funcs, encode='onehot', strategy=strategy),
        RidgeCV(alphas=np.geomspace(1e-3, 1e3, 20))
    )

In [8]:
def make_sl_pipeline(curvature_gamma=1., spline_degree=3, num_funcs=num_funcs):
    spline_n_knots = num_funcs - (spline_degree - 1)
    spline_knots = 'quantile'
    return make_pipeline(
        make_column_transformer(
            (
                SplineTransformer(n_knots=spline_n_knots, knots=spline_knots, degree=spline_degree),
                float_feat_cols
            ),
            (
                StrumLiouvilleTransformer(num_funcs=num_funcs, curvature_spec=CurvatureSpec(alpha=curvature_gamma)),
                int_feat_cols
            )
        ),
        RidgeCV(alphas=np.geomspace(1e-3, 1e3, 20))
    )

In [9]:
pipeline = make_spline_pipeline('uniform').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10895.498751756086

In [10]:
pipeline = make_spline_pipeline('quantile').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10883.391677279977

In [11]:
pipeline = make_bins_pipeline('uniform').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10949.338290115285

In [12]:
pipeline = make_bins_pipeline('quantile').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10882.599855280469

In [13]:
pipeline = make_sl_pipeline().fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10884.906957896515

# HPO

In [29]:
class SplineHpo:
    def search_space(self, trial: optuna.Trial):
        knots = trial.suggest_categorical('knots', ['uniform', 'quantile'])
        degree = trial.suggest_int('degree', 1, 3)
        num_funcs = trial.suggest_int('num_funcs', 1 + degree, 10)
        alpha = trial.suggest_float('alpha', 1e-3, 1e3, log=True)

        return {
            'knots': knots,
            'degree': degree,
            'num_funcs': num_funcs,
            'alpha': alpha
        }

    def pipeline(self, knots, degree, num_funcs, alpha):
        n_knots = num_funcs - (degree - 1)
        return make_pipeline(
            SplineTransformer(n_knots=n_knots, knots=knots, degree=degree),
            Ridge(alpha=alpha)
        )

In [30]:
class BinsHPO:
    def search_space(self, trial: optuna.Trial):
        strategy = trial.suggest_categorical('strategy', ['uniform', 'quantile'])
        num_funcs = trial.suggest_int('num_funcs', 2, 10)
        alpha = trial.suggest_float('alpha', 1e-3, 1e3, log=True)

        return {
            'strategy': strategy,
            'num_funcs': num_funcs,
            'alpha': alpha
        }

    def pipeline(self, strategy, num_funcs, alpha):
        return make_pipeline(
            KBinsDiscretizer(n_bins=num_funcs, encode='onehot', strategy=strategy),
            Ridge(alpha=alpha)
        )

In [31]:
class StrumLiouvilleHPO:
    def search_space(self, trial: optuna.Trial):
        curvature_alpha = trial.suggest_float('curvature_alpha', 0, 2)
        curvature_beta = trial.suggest_float('curvature_beta', 0, 2)
        curvature_center = trial.suggest_float('curvature_center', 0, 1)
        spline_degree = trial.suggest_int('spline_degree', 1, 5)
        knots = trial.suggest_categorical('knots', ['uniform', 'quantile'])
        num_funcs = trial.suggest_int('num_funcs', 1 + spline_degree, 10)
        alpha = trial.suggest_float('alpha', 1e-3, 1e3, log=True)

        return {
            'curvature_alpha': curvature_alpha,
            'curvature_beta': curvature_beta,
            'curvature_center': curvature_center,
            'spline_degree': spline_degree,
            'knots': knots,
            'num_funcs': num_funcs,
            'alpha': alpha
        }

    def pipeline(self, curvature_alpha, curvature_beta, curvature_center, spline_degree, knots, num_funcs, alpha):
        spline_n_knots = num_funcs - (spline_degree - 1)
        return make_pipeline(
            make_column_transformer(
                (
                    SplineTransformer(n_knots=spline_n_knots, knots=knots, degree=spline_degree),
                    float_feat_cols
                ),
                (
                    StrumLiouvilleTransformer(
                        num_funcs=num_funcs,
                        curvature_spec=CurvatureSpec(
                            alpha=curvature_alpha,
                            beta=curvature_beta,
                            center=curvature_center
                        ),
                    ),
                    int_feat_cols
                )
            ),
            Ridge(alpha=alpha)
        )

In [32]:
def hpo(pipeline_hpo, n_trials=500):
    def objective(trial: optuna.Trial):
        params = pipeline_hpo.search_space(trial)
        model = pipeline_hpo.pipeline(**params)
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
        return -np.mean(scores)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_params = study.best_params
    print("Best params:", best_params)
    print("Best CV RMSE:", study.best_value)

    best_model = pipeline_hpo.pipeline(**best_params)
    best_model.fit(X_train, y_train)
    test_rmse = root_mean_squared_error(y_test, best_model.predict(X_test))
    print("Test RMSE:", test_rmse)
    return best_model

In [33]:
hpo(SplineHpo())

[I 2025-11-13 11:08:00,505] A new study created in memory with name: no-name-a3ab3e30-ffcd-457d-bea1-04230b208917


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-11-13 11:08:00,684] Trial 0 finished with value: 11029.266888727198 and parameters: {'knots': 'quantile', 'degree': 1, 'num_funcs': 6, 'alpha': 0.0011806148651394342}. Best is trial 0 with value: 11029.266888727198.
[I 2025-11-13 11:08:00,882] Trial 1 finished with value: 11050.661872767223 and parameters: {'knots': 'quantile', 'degree': 3, 'num_funcs': 6, 'alpha': 0.06176330939088142}. Best is trial 0 with value: 11029.266888727198.
[I 2025-11-13 11:08:01,052] Trial 2 finished with value: 11061.062892624625 and parameters: {'knots': 'uniform', 'degree': 1, 'num_funcs': 7, 'alpha': 19.97991216458179}. Best is trial 0 with value: 11029.266888727198.
[I 2025-11-13 11:08:01,164] Trial 3 finished with value: 11058.133502426062 and parameters: {'knots': 'quantile', 'degree': 1, 'num_funcs': 3, 'alpha': 0.16891950915450069}. Best is trial 0 with value: 11029.266888727198.
[I 2025-11-13 11:08:01,318] Trial 4 finished with value: 11081.55729484598 and parameters: {'knots': 'uniform', '

,steps,"[('splinetransformer', ...), ('ridge', ...)]"
,transform_input,None
,memory,None
,verbose,False
,n_knots,9
,degree,2
,knots,'quantile'
,extrapolation,'constant'
,include_bias,True
,order,'C'
,sparse_output,False


In [34]:
hpo(BinsHPO())

[I 2025-11-13 11:09:58,670] A new study created in memory with name: no-name-403b8ad2-b227-44a8-a6a2-2193deef6d43


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-11-13 11:09:58,838] Trial 0 finished with value: 11099.391212153343 and parameters: {'strategy': 'uniform', 'num_funcs': 9, 'alpha': 321.3672313734318}. Best is trial 0 with value: 11099.391212153343.
[I 2025-11-13 11:09:58,974] Trial 1 finished with value: 11024.653379883446 and parameters: {'strategy': 'quantile', 'num_funcs': 5, 'alpha': 0.48180992873368206}. Best is trial 1 with value: 11024.653379883446.
[I 2025-11-13 11:09:59,110] Trial 2 finished with value: 11023.885552057018 and parameters: {'strategy': 'quantile', 'num_funcs': 5, 'alpha': 190.5807872041258}. Best is trial 2 with value: 11023.885552057018.
[I 2025-11-13 11:09:59,217] Trial 3 finished with value: 11104.476122650114 and parameters: {'strategy': 'uniform', 'num_funcs': 3, 'alpha': 107.3333762036971}. Best is trial 2 with value: 11023.885552057018.
[I 2025-11-13 11:09:59,350] Trial 4 finished with value: 11098.862144385766 and parameters: {'strategy': 'uniform', 'num_funcs': 6, 'alpha': 111.72298247015834}

,steps,"[('kbinsdiscretizer', ...), ('ridge', ...)]"
,transform_input,None
,memory,None
,verbose,False
,n_bins,8
,encode,'onehot'
,strategy,'quantile'
,quantile_method,'warn'
,dtype,None
,subsample,200000
,random_state,None


In [35]:
hpo(StrumLiouvilleHPO())

[I 2025-11-13 11:11:14,885] A new study created in memory with name: no-name-37f01e7e-7f25-4eeb-a930-72eb31d83ea3


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-11-13 11:11:15,632] Trial 0 finished with value: 11036.452016354076 and parameters: {'curvature_alpha': 0.7724623660082688, 'curvature_beta': 1.9952894586346024, 'curvature_center': 0.7718231379813274, 'spline_degree': 3, 'knots': 'quantile', 'num_funcs': 6, 'alpha': 7.032148385346147}. Best is trial 0 with value: 11036.452016354076.
[I 2025-11-13 11:11:16,266] Trial 1 finished with value: 11053.760726560778 and parameters: {'curvature_alpha': 0.26977444785401405, 'curvature_beta': 1.8260613557038918, 'curvature_center': 0.277969678753197, 'spline_degree': 4, 'knots': 'uniform', 'num_funcs': 5, 'alpha': 0.16557464528678414}. Best is trial 0 with value: 11036.452016354076.
[I 2025-11-13 11:11:17,137] Trial 2 finished with value: 11083.464347613528 and parameters: {'curvature_alpha': 0.269120722039041, 'curvature_beta': 0.26782849881713244, 'curvature_center': 0.2427447752251961, 'spline_degree': 4, 'knots': 'uniform', 'num_funcs': 7, 'alpha': 204.92057417116743}. Best is trial 0

,steps,"[('columntransformer', ...), ('ridge', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('splinetransformer', ...), ('strumliouvilletransformer', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
